In [ ]:
%reload_ext autoreload
%autoreload 2

# db: simple read test
import json
import yaml
import types
import open_clip
import torch
import cv2
import numpy as np
import matplotlib as mpl
from scipy.spatial import KDTree
from icecream import ic
import time

from utils.db_utils import get_df, get_data, connect_db, DB
from utils.plotly_utils import *
from utils.vis import *
from utils.predict_scenegraph import PredictSceneGraph
from utils.imagine_nav_planner import ImagineNavPlanner
from utils.scene_graph_utils import update_region
from utils.scene_graph_utils import detect_match
from experiments.test_scenegraph_offline import eval_scenegraph
from constants import *

dump_folder = './dump/prediction_may13/'
output_folder = f'{dump_folder}/objectnav-dino'

# list db size
! ls -lh $output_folder

# load results
results = get_df(f'{output_folder}/result.db', 'result')
print(f'Loaded {len(results)} results')
print(f'Current success rate: {results.tail(1)["success"].values[0]/len(results):.2%}')
print(f'Current SPL: {results["spl"].mean():.2f}')


In [ ]:
# access db
import os
import pathlib
with DB(f'{output_folder}/result.db') as con:
    table = con.table('result')
    print(table)
# episode infos
# steps_df = get_df(f'{output_folder}/result.db', 'result', select=['count_steps', 'episode', 'target', 'habitat_success', 'switch_upstair_count', 'switch_downstair_count'])
steps_df = get_df(f'{output_folder}/result.db', 'result', filter=lambda x:x['count_steps']>490, select=['count_steps', 'episode', 'target', 'habitat_success', 'switch_upstair_count', 'switch_downstair_count'])
print(steps_df.head(30))
# step infos
sample_episode_label = steps_df['episode'].values[0]
with DB(f'{output_folder}/steps/{sample_episode_label}.db') as con:
    table = con.table('step_data')
    print(table)

In [6]:
# load agent modules
device = torch.device("cuda")
args = types.SimpleNamespace(**json.load(open(f'{dump_folder}/args.json')))
with open(f'{dump_folder}/{args.exp_config}') as f:
    exp_config = yaml.safe_load(f)
clip_model, _, clip_preprocess = open_clip.create_model_and_transforms(
    "ViT-H-14", "laion2b_s32b_b79k"
)
clip_model = clip_model.to(device).half()
clip_tokenizer = open_clip.get_tokenizer("ViT-H-14")
clip_model_list = (clip_model, clip_preprocess, clip_tokenizer)
exp_config['utility']['object_correlation'] = 'llm'
exp_config['utility']['region_correlation'] = 'text'

imagine_nav_planner = ImagineNavPlanner(args, exp_config, clip_model_list)
scene_graph = imagine_nav_planner.scene_graph

In [4]:
def get_sg_data(episode_label, step=None):
    if step is None:
        filter = lambda x: (x['episode_label']==episode_label)
    else:
        filter = lambda x: (x['episode_label']==episode_label) & (x['step']==step)
    try:
        data = get_data(
            f'{output_folder}/steps/{episode_label}.db',
            'step_data',
            filter=filter,
            select=[
                'global_scene_graph_pickle',
                'gt_scenegraph',
                'timestamp',
                'step',
                'cate_object',
                'origins_grid',
                'current_grid_pose',
                'camera_position_tensor',
                #### 
                # 'global_bev_rgb_map_tensor',
                # 'annotated_image_tensor',
                # 'gradient_map_tensor',
                # 'color_image_tensor',
                # 'object_nodes_pickle',
            ]
        )
    except Exception as e:
        print(e)
        return None
    # data = data[np.argmax([x['timestamp'] for x in data])]
    return data

In [ ]:
# test clip reigon

from collections import Counter
# get an image for a region
def update_image_idx(self, region):
    objects = region['objects']
    image_coverage = Counter()
    for obj in objects:
        for i,idx in enumerate(obj['image_idx']):
            image_coverage[idx] += obj['conf'][i]
    region['image_idx'] = image_coverage.most_common(1)[0][0]

def get_region_vis_feat(self, region, image_history, topk=1):
    objects = region['objects']
    unique_obj_count = {}
    for obj in objects:
        for i, idx in enumerate(obj['image_idx']):
            unique_obj_count.setdefault(idx, {})
            unique_obj_count[idx].setdefault(obj['caption'], []).append(obj['conf'][i])
    image_coverage = Counter()
    for idx in unique_obj_count:
        for obj, confs in unique_obj_count[idx].items():
            image_coverage[idx] += max(confs)
    region['image_coverage'] = image_coverage
    region['unique_obj_count'] = unique_obj_count
    vis_feat = None
    coverage_sum = 0
    for image_idx, coverage in image_coverage.most_common(topk):
        image = image_history[image_idx]
        if vis_feat is None:
            vis_feat = coverage*self.get_vis_feat(image)
        else:
            vis_feat += self.get_vis_feat(image)
        coverage_sum += coverage
    vis_feat /= coverage_sum
    region['idx_list'] = list(image_coverage.keys())
    region['coverages'] = list(image_coverage.values())
    return vis_feat

def generate_region_caption_TopK_clip(self, scene_graph, image_history, topk=3, img_topk=3):
    for room in scene_graph['rooms']:
        for region in room['regions']:
            similarities = []
            vis_feat = get_region_vis_feat(self, region, image_history, topk=img_topk)
            for caption in region_captions:
                similarities.append(self.get_vis_sim_score(vis_feat, caption))
            captions = sorted(zip(similarities, region_captions), key=lambda x: x[0], reverse=True)[:topk]
            region['caption'] = {}
            region['clip_feat'] = vis_feat
            for similarity, caption in captions:
                region['caption'][caption] = similarity
            region['corr_score'] = self.get_vis_sim_score(vis_feat, self.target)
    return scene_graph

episode_label = steps_df['episode'].iloc[3]
data = get_sg_data(episode_label)

step = data[490]
image_history = [x['color_image'] for x in data]
grid_size = args.map_resolution
origins_grid = step['origins_grid']
bev_map = step['global_bev_rgb_map']
obs_sg = step['global_scene_graph']

scene_graph.set_target(step['cate_object'])
scene_graph.scene_graph = obs_sg

gt_sg = step['gt_scenegraph']
obs_regions = [update_region(region, grid_size, origins_grid) for room in obs_sg['rooms'] for region in room['regions']]
gt_regions = [update_region(region, grid_size, origins_grid) for room in gt_sg['floors'] for region in room['regions']]
obs_objects = [obj for room in obs_sg['rooms'] for region in room['regions'] for obj in region['objects']   ]
gt_objects = [obj for room in gt_sg['floors'] for region in room['regions'] for obj in region['objects']]
start_time = time.time()  
obs_sg = generate_region_caption_TopK_clip(scene_graph, obs_sg, image_history)
print(f'Time taken for region caption: {time.time() - start_time}')

# match region to gt
# matches, metric = eval_scenegraph(clip_model_list, obs_regions, gt_regions, obs_objects, gt_objects)
# for match in matches['region_precision_relaxed']:
#     match['q']['gt_caption'] = match['k']['caption']

# calculate region precision relaxed
matches, score = detect_match(
    clip_model_list=clip_model_list,
    keys=gt_regions,
    queries=obs_regions,
    knn=3,
    overlap_relaxed=True,
    corr_score=None,
    topk=2
)
for match in matches:
    match['q']['gt_caption'] = match['k']['caption']
print(f'region precision relaxed: {score}')

# calculate region recall relaxed
region_recall_relaxed_matches, region_recall_relaxed = detect_match(
    clip_model_list=clip_model_list,
    keys=obs_regions,
    queries=gt_regions,
    knn=3,
    overlap_relaxed=True,
    corr_score=None,
    topk=2
)
print(f'region recall relaxed: {score}')


ic(step['cate_object'])
ic(len(obs_regions))
# ic(obs_regions[10])
for region in obs_regions:
    print(f'region {region["id"]}: images= {list(zip(region["idx_list"], region["coverages"]))}')
    print(f'caption: {region["caption"]}')
    print(f'gt_caption: {region["gt_caption"]}')
    # print(region['image_coverage'])
    # print(region['unique_obj_count'])
    # for obj in region['objects']:
    #     print(obj['id'], obj['caption'], obj['image_idx'])
    print('--------------------------------')
# ic(step['object_nodes'])

In [ ]:
len([obj for room in obs_sg['rooms'] for region in room['regions'] for obj in region['objects']])

In [ ]:
region_id = 26
frame_id = obs_regions[region_id]['idx_list'][0]
show_image(data[frame_id]['annotated_image'][...,::-1])
show_image(data[frame_id]['color_image'])

In [ ]:
def get_image_corr(vis_feat, target):
    return imagine_nav_planner.scene_graph.get_vis_sim_score(vis_feat, target)
# vis_feat = imagine_nav_planner.scene_graph.get_vis_feat(data[frame_id]['color_image'])
vis_feat = imagine_nav_planner.scene_graph.get_vis_feat(cv2.imread('img/image.png')[...,::-1])
# vis_feat = get_region_vis_feat(scene_graph, obs_regions[region_id], image_history, topk=1)

# imagine_nav_planner.scene_graph.get_vis_sim_score(vis_feat, step['cate_object'])
# target_list = categories_21_plus_stairs#+['somewhere near '+x for x in categories_21_plus_stairs]
# target_list = ['stairs', 'not stairs', 'somewhere near stairs', 'somewhere not near stairs']
target_list = region_captions
# target_list = detector_classes
score_list = []
for target in target_list:
    score_list.append(get_image_corr(vis_feat, target))
top5_result = sorted(zip(score_list, target_list), key=lambda x: x[0], reverse=True)[:5]
# print top 5 with score and target
ic(top5_result)


In [ ]:
# test text sim range
# print(scene_graph.get_text_sim_score('guest bedroom', 'bed'))
# print(scene_graph.get_text_sim_score('guest bedroom', 'toilet'))
# print(scene_graph.get_text_sim_score('guest bedroom', 'tv_monitor'))
# print(scene_graph.get_text_sim_score('guest bedroom', 'sofa'))
# print(scene_graph.get_text_sim_score('guest bedroom', 'chair'))
# print(scene_graph.get_text_sim_score('guest bedroom', 'plant'))

# print(scene_graph.get_text_sim_score('bed', 'bed'))
# print(scene_graph.get_text_sim_score('bed', 'toilet'))
# print(scene_graph.get_text_sim_score('bed', 'tv_monitor'))
# print(scene_graph.get_text_sim_score('bed', 'sofa'))
# print(scene_graph.get_text_sim_score('bed', 'chair'))
# print(scene_graph.get_text_sim_score('bed', 'plant'))


print(scene_graph.get_text_sim_score('living room', 'sofa'))
print(scene_graph.get_text_sim_score('living room', 'stairs'))
print(scene_graph.get_text_sim_score('living room', 'tv_monitor'))
print(scene_graph.get_text_sim_score('bedroom', 'toilet'))

print(scene_graph.get_text_sim_score('stair hall', 'stairs'))
print(scene_graph.get_text_sim_score('study room', 'stairs'))
print(scene_graph.get_text_sim_score('living room', 'stairs'))
print(scene_graph.get_text_sim_score('kitchen', 'stairs'))
print(scene_graph.get_text_sim_score('hallway', 'stairs'))

In [ ]:
obs_regions[region_id]['caption']

In [ ]:
caption_id = np.argmax([obs_regions[region_id]['caption'][caption] for caption in ])

In [ ]:
fig = create_fig(img=bev_map[...,::-1])
plot_region(fig, obs_regions, 480, show_objects=True)
fig.show()

fig = create_fig(img=bev_map[...,::-1])
plot_region(fig, gt_regions, 480, show_objects=True)
fig.show()

In [ ]:
# print all region captions
gt_region_captions = set()
for i in range(len(results)):
    episode_label = results['episode'].iloc[i]
    print(f'Processing episode {episode_label}')
    data = get_sg_data(episode_label)
    if data is None:
        print(f'No data for episode {episode_label}')
        continue
    count_episode += 1
    grid_size = args.map_resolution
    step = data[-1]
    origins_grid = step['origins_grid']
    gt_sg = step['gt_scenegraph']
    gt_regions = [update_region(region, grid_size, origins_grid) for room in gt_sg['floors'] for region in room['regions']]
    for room in gt_sg['floors']:
        for region in room['regions']:
            gt_region_captions.add(region['caption'])

gt_region_captions_map = {}
region_captions = set()
for region_caption in gt_region_captions:
    merged_region_caption = region_caption
    if 'bath' in region_caption or 'shower' in region_caption or 'wash' in region_caption:
        merged_region_caption = 'bathroom'
    elif 'powder' in region_caption or 'nursery' in region_caption or 'spa' in region_caption:
        merged_region_caption = 'bathroom'
    elif 'bedroom' in region_caption:
        merged_region_caption = 'bedroom'
    elif 'hallway' in region_caption:
        merged_region_caption = 'hallway'
    elif 'study' in region_caption:
        merged_region_caption = 'study room'
    elif 'laundry' in region_caption:
        merged_region_caption = 'laundry room'
    elif 'lounge' in region_caption:
        merged_region_caption = 'living room'
    elif 'kitchen' in region_caption:
        merged_region_caption = 'kitchen'
    elif 'dining' in region_caption:
        merged_region_caption = 'dining room'
    elif 'guest' in region_caption:
        merged_region_caption = 'bedroom'
    elif 'office' in region_caption:
        merged_region_caption = 'study room'
    elif 'pantry' in region_caption:
        merged_region_caption = 'storage'
    elif 'shower' in region_caption:
        merged_region_caption = 'bathroom'
    elif 'storage' in region_caption:
        merged_region_caption = 'storage'
    elif 'music' in region_caption or 'media' in region_caption or 'rec' in region_caption or 'entertainment' in region_caption:
        merged_region_caption = 'study room'
    elif 'stair' in region_caption:
        merged_region_caption = 'stair hall'
    elif 'family' in region_caption:
        merged_region_caption = 'living room'
    elif 'utility' in region_caption:
        merged_region_caption = 'storage'
    elif 'closet' in region_caption:
        merged_region_caption = 'wardrobe area'
    elif 'toilet' in region_caption:
        merged_region_caption = 'bathroom'
    elif 'corridor' in region_caption:
        merged_region_caption = 'hallway'
    elif 'attic access' in region_caption:
        merged_region_caption = 'stair hall'
    region_captions.add(merged_region_caption)
    gt_region_captions_map[region_caption] = merged_region_caption
print(region_captions)
ic(gt_region_captions_map)

In [16]:
# gt co-occurrence (obj-obj, region-obj)
from collections import Counter

def get_sg_data(episode_label, step=None):
    if step is None:
        filter = lambda x: (x['episode_label']==episode_label) & (x['step']%5==0)
    else:
        filter = lambda x: (x['episode_label']==episode_label)
    try:
        data = get_data(
            f'{output_folder}/steps/{episode_label}.db',
            'step_data',
            filter=filter,
            select=[
                'global_scene_graph_pickle',
                'gt_scenegraph',
                'timestamp',
                'step',
                'cate_object',
                'origins_grid',
                'current_grid_pose',
                'camera_position_tensor',
                #### 
                # 'global_bev_rgb_map_tensor',
                # 'annotated_image_tensor',
                # 'gradient_map_tensor',
                # 'color_image_tensor',
                # 'object_nodes_pickle',
            ]
        )
    except Exception as e:
        print(e)
        return None
    # data = data[np.argmax([x['timestamp'] for x in data])]
    return data


obj_to_obj_co_occurrence = {}
region_to_obj_co_occurrence = {}
count_episode = 0
# for i in range(1):
for i in range(len(results)):
    episode_label = results['episode'].iloc[i]
    print(f'Processing episode {episode_label}')
    data = get_sg_data(episode_label)
    if data is None:
        print(f'No data for episode {episode_label}')
        continue
    count_episode += 1
    grid_size = args.map_resolution
    step = data[-1]
    origins_grid = step['origins_grid']
    gt_sg = step['gt_scenegraph']
    gt_regions = [update_region(region, grid_size, origins_grid) for room in gt_sg['floors'] for region in room['regions']]
    for region in gt_regions:
        object_counter = Counter()
        for obj in region['objects']:
            object_counter[obj['caption']] += 1
        for obj, count in object_counter.items():
            region_caption = gt_region_captions_map[region['caption']]
            region_to_obj_co_occurrence.setdefault(region_caption, {})
            region_to_obj_co_occurrence[region_caption].setdefault(obj, 0)
            region_to_obj_co_occurrence[region_caption][obj] += count
        for i, (obj, count) in enumerate(object_counter.items()):
            obj_to_obj_co_occurrence.setdefault(obj, {})
            # increase the count for every other object in the region
            for j, (other_obj, other_count) in enumerate(object_counter.items()):
                if i != j:
                    obj_to_obj_co_occurrence[obj].setdefault(other_obj, 0)
                    obj_to_obj_co_occurrence[obj][other_obj] += count
print(f'Processed {count_episode} episodes')
ic(obj_to_obj_co_occurrence)
ic(region_to_obj_co_occurrence)

In [ ]:
region_near_region_co_occurrence = {}
near_region_knn = 3
near_region_radius = 60
count_episode = 0
for i in range(len(results)):
    episode_label = results['episode'].iloc[i]
    print(f'Processing episode {episode_label}')
    data = get_sg_data(episode_label)
    if data is None:
        print(f'No data for episode {episode_label}')
        continue
    count_episode += 1
    grid_size = args.map_resolution
    step = data[-1]
    origins_grid = step['origins_grid']
    gt_sg = step['gt_scenegraph']
    gt_regions = [update_region(region, grid_size, origins_grid) for room in gt_sg['floors'] for region in room['regions']]
    tree = KDTree([region['center'] for region in gt_regions])
    for region in gt_regions:
        distances, indices = tree.query([region['center']], k=near_region_knn+1)
        if not isinstance(distances, np.ndarray):
            distances, indices = np.array([distances]), np.array([indices])
        for distance, index in zip(distances[0], indices[0]):
            near_region = gt_regions[index]
            if region['id']==near_region['id']:
                continue
            if distance<near_region_radius:
                region_caption = gt_region_captions_map[region['caption']]
                near_region_caption = gt_region_captions_map[near_region['caption']]
                region_near_region_co_occurrence.setdefault(region_caption, {})
                region_near_region_co_occurrence[region_caption].setdefault(near_region_caption, 0)
                region_near_region_co_occurrence[region_caption][near_region_caption] += 1

print(f'Processed {count_episode} episodes')
ic(region_near_region_co_occurrence)

In [22]:
# save co-occurrence
np.save('tools/obj_co_occurrence.npy', obj_to_obj_co_occurrence)
np.save('tools/region_co_occurrence.npy', region_to_obj_co_occurrence)
np.save('tools/region_region_co_occurrence.npy', region_near_region_co_occurrence)


In [7]:
import numpy as np
from utils.plotly_utils import *
from constants import *
obj_to_obj_co_occurrence = np.load('tools/obj_co_occurrence.npy', allow_pickle=True).tolist()
region_to_obj_co_occurrence = np.load('tools/region_co_occurrence.npy', allow_pickle=True).tolist()
region_near_region_co_occurrence = np.load('tools/region_region_co_occurrence.npy', allow_pickle=True).tolist()

In [ ]:
# plot co-occurrence
n_obj = len(categories_21_plus_stairs)
n_region = len(region_captions)
heatmap_obj_obj = np.zeros((n_obj, n_obj))
for obj_i in obj_to_obj_co_occurrence:
    for obj_j in obj_to_obj_co_occurrence[obj_i]:
        heatmap_obj_obj[obj_category_to_idx[obj_i], obj_category_to_idx[obj_j]] = obj_to_obj_co_occurrence[obj_i][obj_j]

heatmap_region_obj = np.zeros((n_region, n_obj))
for region_i in region_to_obj_co_occurrence:
    if region_i=='UNKNOWN':
        continue
    for obj_j in region_to_obj_co_occurrence[region_i]:
        heatmap_region_obj[region_caption_to_idx[region_i], obj_category_to_idx[obj_j]] = region_to_obj_co_occurrence[region_i][obj_j]
n_obj_detector = len(detector_classes)

heatmap_region_region = np.zeros((n_region, n_region))
for region_i in region_near_region_co_occurrence:
    for region_j in region_near_region_co_occurrence[region_i]:
        if region_i=='UNKNOWN' or region_i==region_j:
            continue
        heatmap_region_region[region_caption_to_idx[region_i], region_caption_to_idx[region_j]] = region_near_region_co_occurrence[region_i][region_j]

# normalize
heatmap_obj_obj_norm = heatmap_obj_obj.copy().clip(max=764)
heatmap_obj_obj_norm -= heatmap_obj_obj_norm.min()
heatmap_obj_obj_norm /= heatmap_obj_obj_norm.max()
heatmap_obj_obj_norm = heatmap_obj_obj_norm*0.8+0.2
heatmap_region_obj_norm = heatmap_region_obj.copy().clip(max=266)
heatmap_region_obj_norm -= heatmap_region_obj_norm.min()
heatmap_region_obj_norm /= heatmap_region_obj_norm.max()
heatmap_region_obj_norm = heatmap_region_obj_norm*0.8+0.2
# print(heatmap_obj_obj_norm.max(), heatmap_obj_obj_norm.min())
# print(heatmap_region_obj_norm.max(), heatmap_region_obj_norm.min())


# plot scores
fig = create_fig(600, 600)
plot_heatmap(fig, heatmap_obj_obj_norm, 'obj-obj co-occurrence', x=categories_21_plus_stairs, y=categories_21_plus_stairs)
fig.show()
fig = create_fig(600, 600)
plot_heatmap(fig, heatmap_region_obj_norm, 'region-obj co-occurrence', x=categories_21_plus_stairs, y=gt_region_captions)
fig.show()
fig = create_fig(600, 600)
plot_heatmap(fig, heatmap_region_region, 'region-region co-occurrence', x=gt_region_captions, y=gt_region_captions)
fig.show()


In [ ]:
heatmap_region_obj

In [ ]:
n_target = len(category_to_id)
n_obj = len(categories_21_plus_stairs)
heatmap_obj_target = np.zeros((n_obj, n_target))
for i, obj in enumerate(categories_21_plus_stairs):
    for j, target in enumerate(category_to_id):
        heatmap_obj_target[i, j] = heatmap_obj_obj_norm[categories_21_plus_stairs.index(obj), categories_21_plus_stairs.index(target)]

heatmap_region_target = np.zeros((n_region, n_target))
for i, region in enumerate(region_captions):
    for j, target in enumerate(category_to_id):
        heatmap_region_target[i, j] = heatmap_region_obj_norm[region_captions.index(region), categories_21_plus_stairs.index(target)]


# plot co-occurrence
fig = create_fig(600, 600)
plot_heatmap(fig, heatmap_obj_target, 'obj-obj co-occurrence', x=category_to_id, y=categories_21_plus_stairs)
fig.show()
fig = create_fig(600, 600)
plot_heatmap(fig, heatmap_region_target, 'region-obj co-occurrence', x=category_to_id, y=gt_region_captions)
fig.show()


In [ ]:
chance_regions = ['bathroom', 'kitchen', 'bedroom', 'dining room', 'living room', 'study room']
heatmap_region_important = np.zeros((n_region, len(chance_regions)))
for i, region in enumerate(region_captions):
    for j, chance_region in enumerate(chance_regions):
        heatmap_region_important[i, j] = heatmap_region_region[region_captions.index(region), region_captions.index(chance_region)]

heatmap_region_possibility = heatmap_region_important/heatmap_region_important.sum(axis=0)
fig = create_fig(600, 600)
plot_heatmap(fig, np.where((heatmap_region_important>5) & (heatmap_region_possibility>0.1), heatmap_region_possibility, 0), 'region-region co-occurrence', x=chance_regions, y=gt_region_captions)
fig.show()

In [ ]:
import peek
hint_pairs = {}
hint_list = []
# get i,j index of heatmap_region_important>15
high_freq_mask = (heatmap_region_important>5) & (heatmap_region_possibility>0.1)
for i, j in zip(*np.where(high_freq_mask)):
    pair = (chance_regions[j], gt_region_captions[i])
    possibility = heatmap_region_possibility[i, j]
    hint_pairs.setdefault(pair[0], []).append((pair[1], possibility))
for k,v in hint_pairs.items():
    v.sort(key=lambda x: x[1], reverse=True)
    hint_pairs[k] = v[:4]
    near_regions = ", ".join([f'{x[0]} ({x[1]*100:.0f}%)' for x in v])
    hint_list.append(f'- {k} is usually near {near_regions}.')
# print hints
print(',\n'.join([f'"{x}"' for x in hint_list]))


In [ ]:
import peek
hint_pairs = {}
hint_list = []
# get i,j index of heatmap_region_important>15
low_freq_mask = (heatmap_region_important<5) & (heatmap_region_important>0)
for i, j in zip(*np.where(low_freq_mask)):
    pair = (chance_regions[j], gt_region_captions[i])
    possibility = heatmap_region_possibility[i, j]
    hint_pairs.setdefault(pair[0], []).append((pair[1], possibility))
for k,v in hint_pairs.items():
    v.sort(key=lambda x: x[1], reverse=True)
    hint_pairs[k] = v[:4]
    near_regions = ", ".join([f'{x[0]}' for x in v])
    hint_list.append(f'- {k} is usually NOT near {near_regions}.')
# print hints
print(',\n'.join([f'"{x}"' for x in hint_list]))


In [ ]:
np.save('tools/heatmap_obj_obj.npy', heatmap_obj_obj_norm)
np.save('tools/heatmap_region_obj.npy', heatmap_region_obj_norm)

In [7]:
heatmap_obj_obj = np.load('tools/heatmap_obj_obj.npy', allow_pickle=True)
heatmap_region_obj = np.load('tools/heatmap_region_obj.npy', allow_pickle=True)

In [ ]:
def get_cooccurrence_object(obj_label, target_label):
    if obj_label == target_label:
        return 1
    elif obj_label in categories_21_plus_stairs and target_label in categories_21_plus_stairs:
        return heatmap_obj_obj[categories_21_plus_stairs.index(obj_label), categories_21_plus_stairs.index(target_label)]
    else:
        return 0

def get_cooccurrence_region(region_label, target_label):
    if region_label in region_captions and target_label in categories_21_plus_stairs:
        return heatmap_region_obj[region_captions.index(region_label), categories_21_plus_stairs.index(target_label)]
    else:
        return 0

print(get_cooccurrence_object('bed', 'toilet'))
print(get_cooccurrence_object('bed', 'sofa'))
print(get_cooccurrence_object('bed', 'bed'))
print(get_cooccurrence_region('bedroom', 'sofa'))
print(get_cooccurrence_region('bathroom', 'sofa'))
print(get_cooccurrence_region('living room', 'sofa'))

In [ ]:
region_to_obj_co_occurrence

In [ ]:
obj_to_obj_co_occurrence

In [ ]:
# co_occur_mtx = np.load('tools/obj.npy')
# co_occur_mtx -= co_occur_mtx.min()
# co_occur_mtx /= co_occur_mtx.max() 

# co_occur_room_mtx = np.load('tools/room.npy')
# co_occur_room_mtx -= co_occur_room_mtx.min()
# co_occur_room_mtx /= co_occur_room_mtx.max()

# fig = create_fig(600, 600)
# plot_heatmap(fig, co_occur_mtx, 'llm obj-obj co-occurrence', x=categories_21_plus_stairs, y=categories_21_plus_stairs)
# fig.show()
# fig = create_fig(600, 600)
# plot_heatmap(fig, co_occur_room_mtx, 'llm region-obj co-occurrence', x=categories_21_plus_stairs, y=gt_region_captions)
# fig.show()


In [ ]:
# get all gt region captions
gt_region_captions = set()

for i in range(len(results)):
    episode_label = results['episode'].iloc[i]
    data = get_sg_data(episode_label)
    if data is None:
        print(f'No data for episode {episode_label}')
        continue
    grid_size = args.map_resolution
    origins_grid = step['origins_grid']
    gt_sg = step['gt_scenegraph']
    gt_regions = [update_region(region, grid_size, origins_grid) for room in gt_sg['floors'] for region in room['regions']]
    for region in gt_regions:
        gt_region_captions.add(region['caption'])
gt_region_captions

In [ ]:
# gt co-occurrence (obj-obj, region-obj)
from collections import Counter

def get_sg_data(episode_label, step=None):
    if step is None:
        filter = lambda x: (x['episode_label']==episode_label) & (x['step']%5==0)
    else:
        filter = lambda x: (x['episode_label']==episode_label)
    try:
        data = get_data(
            f'{output_folder}/steps/{episode_label}.db',
            'step_data',
            filter=filter,
            select=[
                'global_scene_graph_pickle',
                'gt_scenegraph',
                'timestamp',
                'step',
                'cate_object',
                'origins_grid',
                'current_grid_pose',
                'camera_position_tensor',
                'frontier_scores_pickle'
            ]
        )
    except Exception as e:
        print(e)
        return None
    # data = data[np.argmax([x['timestamp'] for x in data])]
    return data


count_episode = 0
frontier_scores = {'baseline': [], 'exploration': [], 'exploitation': [], 'distance': [], 'final': []}
for i in range(len(results)):
    episode_label = results['episode'].iloc[i]
    # print(f'Processing episode {episode_label}')
    data = get_sg_data(episode_label)
    if data is None:
        print(f'No data for episode {episode_label}')
        continue
    count_episode += 1
    for step in data:
        if not 'frontier_scores' in step:
            continue
        for key in step['frontier_scores']:
            frontier_scores[key].extend(step['frontier_scores'][key])
print(f'Processed {count_episode} episodes')
for key in frontier_scores:
    print(key, np.mean(frontier_scores[key]), np.min(frontier_scores[key]), np.max(frontier_scores[key]))

In [ ]:
# sum non-zero
scores = np.array(frontier_scores['exploitation'])
np.min(scores[scores>0])

In [ ]:
len(data)